## Język Python - Laboratorium 4.

## ZADANIE 1

Napisz manager kontekstu, który zapisze do pliku podanego w argumencie czas wykonywania się tego bloku kodu. Przydatny będzie moduł `time` oraz funkcja `open`

In [ ]:
from time import sleep, time
import os

class LoggingTime:    
    pass

for valid in range(6):
    
    with LoggingTime('tmp.txt'):
        sleep(valid)
        
    with open('tmp.txt') as fh:
        value = float(fh.read())
        print('Took %fs, should take %ds, is it ok? %s' % (value, valid, round(value) == round(valid)))
        
    os.remove('tmp.txt')

## ZADANIE 2
Napisz manager kontekstu, który sprawi, że funkcja generująca kolejne liczby fibonnaciego "przesunie" się o zadaną liczbę pozycji na czas kontekstu. Po wyjściu z kontekstu powinna wracać do wyjściowego stanu. Innymi slowy, chcemy móc w obrębie "with" pracować z liczbami fibonnaciego o k dalszymi, ale po skończeniu się "with" chcemy żeby wyjściowa zmienna była nienaruszona. Manager powinien działać z dowolną klasą, która posiada metodę "next_value".

In [ ]:
class Fibonnaci:
    
    def __init__(self):
        self.fib = 0
        self.last = -1           
        
    def next_value(self):
        if self.last == -1:
            self.last = 0
            self.fib = 1
        else:
            self.last, self.fib = self.fib, self.fib + self.last
        return self.fib
    
    def current_value(self):
        return self.fib
    
class Shifted:
    
    pass
    
F = Fibonnaci()
x = F.next_value()

with Shifted(F, 0) as fib:
    for i, v in zip(range(1, 10), [1,  2, 3, 5, 8, 13, 21, 34, 55]):
        f = fib.next_value()
        print("Iteration %d\t Value %d \t Valid %s" % (i, f, f == v))

# Teraz F powinno dalej wskazywac na x
print('Is F still correct? %s' % (x == F.current_value()))

with Shifted(F, 9) as fib:
    for i, v in zip(range(1, 10), [89, 144, 233, 377, 610, 987, 1597, 2584, 4181]):
        f = fib.next_value()
        print("Iteration %d\t Value %d \t Valid %s" % (i, f, f == v))

## ZADANIE 3
Napisz klasę "Bag of words", która będzie "workiem słów" z zadanego dokumentu. Oznacza to, że ma w sobie przechowywać informacje o tym ile razy każde słowo wystąpiło w dokumencie. Dodatkowo ma udostępniać następującą funkcjonalność:

* Można utworzyć go na dwa sposoby:

         bow = BagOfWords("ala ma kota ala ma ala")
         bow = BagOfWords(open("plik.txt"))
         
* Możliwość wyświetlania go po prostu pisząc `print(bow)` (kolejność taka sama jak przy przeglądaniu forem)

        ala:3, ma:2, kota:1

* Sprawdzanie czy jakieś słowo należy do naszego worka pisząc 

        if 'ala' in bow:
            ...
            
* Przeglądanie słów w worku od najczęściej do najrzadziej występującego, czyli

        for word in bow:
            print(word)
     
  powinniśmy zobaczyć 
  
        ala
        ma
        kota
        
* Możliwość dodawania dwóch worków, pisząc

        bow1 = BagOfWords("ala ma kota ala ma ala")
        bow2 = BagOfWords("tomek tez ma kota")
        bow3 = bow1 + bow2
        print('tomek' in bow1) # False
        print('tomek' in bow3) # True
        print('ala' in bow3) # True
        print(bow3) # ala:3, ma:3, kota:2, tez:1, tomek:1
        
* Odczytywanie częstości wystąpień słów poprzez nawiasy kwadratowe 

        print(bow1["ala"]) # 3
        print(bow3["ala"]) # 3

* Aktualizację cześtości wystąpień

        bow3['tomek'] = 10
        for el in bow3:
            print(el)
            
  powinno wyświetlić najpierw "tomek"

In [ ]:
class BagOfWords:
    
    pass

Napisz klasę pochodną o nazwie `SetOfWords`, która zapamiętuje jedynie które słowa wystąpiły, bez zliczania ile razy. Powinno zmienić się wyświetlanie (zamiast 'ala:3, ma:2, kota:1' powinniśmy zobaczyć 'ala, ma, kota'), postaraj się napisać ją jak najwydajniej. Z których elementów obecnych wewnątrz klasy `BagOfWords` możesz zrezygnować? Dlaczego? Jak dużo kodu musisz zmodyfikować by mieć z jednej strony poprawną implementację a z drugiej jak najmniej zmienić klasy bazowej?

In [ ]:
class SetOfWords(BagOfWords):
    
    pass

## Zadanie dodatkowe

Przerób implementację `BagOfWords` na `BagOfPairsOfWords`, gdzie zamiast przechowywać liczność pojedynczych słów, przechowuj jak wiele razy wystąpiły obok siebie pary słów, np.

        "ala ma kota ala ma psa"
        
zawiera następujące pary:

        (None, 'ala'): 1
        ('ala', 'ma'): 2
        ('ma', 'kota'): 1
        ('kota', 'ala'): 1
        ('ma', 'psa'): 1
        ('psa', None): 1
        
Powinny działać wszystkie funkcjonalności `BagOfWords`, po prostu pracujemy na parach, czyli np.

        bopow = BagOfPairsOfWords('ala ma kota ala ma psa')
        bopow[('ala', 'ma')] == 2
        ('ala', 'ma') in bopow
        for word1, word2 in bopow:
            print(word1, word2, bopow[(word1, word2)])
            
Dodatkowo jeśli użytkownik poprosi o słowo, zamiast o parę to powinien dostać wszystkie słowa, z którymi zadane występuje w parze, czyli np.

        for word in bopow['ala']:
            print word, bopow[('ala', word)]
            
wyświetla 

        ma, 2
